In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

/home/rushi/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df =pd.read_csv('/home/rushi/Downloads/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,0:-1].values
y =df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [7]:
X =scaler.fit_transform(X)


In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1) 

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense


In [12]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=20 )

Epoch 1/20
20/20 [==============================] - 1s 2ms/step - loss: 0.7377 - accuracy: 0.4886
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 0.6708 - accuracy: 0.5928
Epoch 3/20
20/20 [==============================] - 0s 2ms/step - loss: 0.6243 - accuracy: 0.6547
Epoch 4/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5901 - accuracy: 0.6792
Epoch 5/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5649 - accuracy: 0.6971
Epoch 6/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 0.7134
Epoch 7/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7199
Epoch 8/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5191 - accuracy: 0.7264
Epoch 9/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5099 - accuracy: 0.7313
Epoch 10/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.7378
Epoch 11/20
20/20 [

In [13]:
# 1. How to select appropriate optimizer
# 2. No.of nodes in the layer
# 3. How to select no.of layers
# 4. All best parameter at once 

In [18]:
import kerastuner as kt

In [27]:
def buildmodel(hp):

    model=Sequential()

    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [28]:
tuner = kt.RandomSearch(buildmodel,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [29]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.3636363744735718

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 01m 42s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [34]:
best_model = tuner.get_best_models(num_models=1)[0]

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                288       
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model.fit(X_train,y_train, batch_size=32, epochs=100, initial_epoch=11,validation_data=(X_test,y_test))

Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4590 - accuracy: 0.7785 - val_loss: 0.4817 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4577 - accuracy: 0.7818 - val_loss: 0.4821 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4565 - accuracy: 0.7818 - val_loss: 0.4817 - val_accuracy: 0.7792
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4551 - accuracy: 0.7818 - val_loss: 0.4814 - val_accuracy: 0.7792
Epoch 16/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4543 - accuracy: 0.7818 - val_loss: 0.4816 - val_accuracy: 0.7792
Epoch 17/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4533 - accuracy: 0.7818 - val_loss: 0.4818 - val_accuracy: 0.7857
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4520 - accuracy: 0.7834 - val_loss: 0.4816 - val_accuracy: 0.7857
Epoch

In [38]:
def build_model(hp):

    model= Sequential()
    units= hp.Int('units',8,128, step=8)

    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

    return model


In [43]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir')

In [44]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 05s


In [45]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}